<a href="https://colab.research.google.com/gist/Balazhir/c26ef637480717b821a18af9192d7e63/untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
set -e

echo "DOWNLOADING REDIST"
apt-get update -y
apt-get install -y --no-install-recommends wget ca-certificates lib32stdc++6 lib32gcc-s1 unzip curl

echo "CREATING FOLDERS"
STEAMCMD_DIR="/root/steamcmd"
RUST_SERVER_DIR="/root/rust_server"
mkdir -p "$STEAMCMD_DIR" "$RUST_SERVER_DIR"

echo "DOWNLOADING STEAMCMD"
cd "$STEAMCMD_DIR"
wget -q https://steamcdn-a.akamaihd.net/client/installer/steamcmd_linux.tar.gz
tar -xzf steamcmd_linux.tar.gz
chmod +x steamcmd.sh

echo "DOWNLOADING RUST DEDICATED SERVER"
./steamcmd.sh +login anonymous +force_install_dir "$RUST_SERVER_DIR" +app_update 258550 validate +quit

echo "SERVER DOWNLOADED $RUST_SERVER_DIR"


In [ ]:
%%bash
set -e

RUST_SERVER_DIR="/root/rust_server"
cd "$RUST_SERVER_DIR"
chmod +x RustDedicated || true

SERVER_PORT=28015

echo "LAUNCHING SERVER"
nohup ./RustDedicated -batchmode -nographics \
  +server.port $SERVER_PORT \
  +server.identity "vanilla" \
  +server.hostname "GITHUB: BALAZHIR" \
  +server.maxplayers 10 \
  > rust.log 2>&1 &

sleep 2

PUBLIC_IP=$(curl -s ifconfig.me)

echo "=========================================="
echo " RUST DEDICATED STARTED!"
echo " IP: $PUBLIC_IP"
echo " PORT: $SERVER_PORT"
echo ""
echo " (Colab not allows to connect to its servers)"
echo "=========================================="
echo ""
echo "SERVER PID:"
pgrep -f RustDedicated || echo "PID NOT FOUND"


In [ ]:
%%bash
set +e

RUST_SERVER_DIR="/root/rust_server"
SERVER_PORT=28015

echo "CHECKING SERVER"
if pgrep -f RustDedicated >/dev/null; then
    echo "✔ SERVER WORKS FINE"
    PID=$(pgrep -f RustDedicated)
    echo " PID: $PID"
else
    echo "✘ SERVER DONT WORK"
fi

echo ""
echo "CHECKING PORT"
if ss -lnt | grep -q ":$SERVER_PORT"; then
    echo "✔ $SERVER_PORT IS RIGHT"
else
    echo "✘ $SERVER_PORT IS NOT RIGHT"
fi

echo ""
echo "LOG:"
tail -n 40 "$RUST_SERVER_DIR/rust.log" || echo "LOG DONT WORK"
